In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, GRU, BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint
import pandas as pd
import numpy as np
import re

Using TensorFlow backend.


In [2]:
#Data retrieved from https://www.kaggle.com/mousehead/songlyrics
song_df = pd.read_csv("data/songlyrics/songdata.csv")
song_df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [3]:
song_df.at[0, "text"]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?  \n  \nAnd when we go for a walk in the park  \nAnd she holds me and squeezes my hand  \nWe'll go on walking for hours and talking  \nAbout all the things that we plan  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?\n\n"

In [4]:
#Use the previous 64 characters to predict the 65th
SEQ_LEN = 64

In [5]:
def clean_lyrics(lyrics):
    lyrics = lyrics.replace("\n", ".").lower() #Newlines generally indicate pauses
    lyrics = re.sub(r"\(.*\)", "", lyrics) #Get rid of lines inside parentheses (chorus)
    lyrics = re.sub(r"\[.*\]", "", lyrics) #Get rid of lines inside brackets [chorus]
    lyrics = re.sub(r"[\(\)\[\]]", "", lyrics) #Some parentheses were unbalanced...
    lyrics = re.sub(r"(\s+\.)+", ". ", lyrics) #Some brackets were unbalanced...
    lyrics = re.sub(r"([\?\.\!\;\,])\.+", r"\1", lyrics)  #Drop periods appearing after other punctuation
    lyrics = re.sub(r"\s+", " ", lyrics)  #Replace 1 or more whitespace characters with a single space
    return " " * (SEQ_LEN) + lyrics + "E" #Pad the beginning with whitespace so we can predict without feeding in lyrics

In [6]:
#Check out random songs to see if we should add anything to clean_lyrics
random_index = np.random.choice(len(song_df))
clean_lyrics(song_df.at[random_index, "text"])

"                                                                you got to learn how to fall. before you learn to fly. and mama, mama it ain't no lie. before you learn to fly. learn how to fall. you got to drift in the breeze. before you set your sails. it's an occupation where the wind prevails. before you set your sails. drift in the breeze. oh and it's the same old story. ever since the world began. everybody got the runs for glory. nobody stop and scrutinize the plan. nobody stop and scrutinize the plan. you got to learn how to fall. before you learn to fly. the tank towns they tell no lie. before you learn to fly. learn how to fall.E"

In [7]:
#Vectorize clean_lyrics over the entire song text column
song_df["clean"] = song_df.text.apply(clean_lyrics)

In [8]:
song_df.head()

,artist,song,link,text,clean
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...",...
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...",...
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,...


In [9]:
data = song_df.clean.values
data[0]

"                                                                look at her face, it's a wonderful face. and it means something special to me. look at the way that she smiles when she sees me. how lucky can one fellow be? she's just my kind of girl, she makes me feel fine. who could ever believe that she could be mine? she's just my kind of girl, without her i'm blue. and if she ever leaves me what could i do, what could i do? and when we go for a walk in the park. and she holds me and squeezes my hand. we'll go on walking for hours and talking. about all the things that we plan. she's just my kind of girl, she makes me feel fine. who could ever believe that she could be mine? she's just my kind of girl, without her i'm blue. and if she ever leaves me what could i do, what could i do?E"

In [10]:
from itertools import chain

#Chain takes a bunch of iterables and connects them together
#The * unpacks an iterable so you can use it as positional arguments
#For example:  print(*[1,2,3]) is the same as calling print(1,2,3)
char_set = set(chain(*data))
len(char_set) #46 characters to predict

46

In [11]:
print(sorted(char_set))

[' ', '!', '"', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'E', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [12]:
len(list(chain(*data))) #56 million characters in data

56358466

In [13]:
N = len(data) #Number of songs
K = len(char_set) #Number of unique characters

In [14]:
#Mappings back and forth between character and integer index
#It is imperative to sort the char_set, otherwise the enumeration will
#return different indices in future sessions, which will ruin our model
letter2idx = dict((c, i) for i, c in enumerate(sorted(char_set)))
idx2letter = dict((i, c) for i, c in enumerate(sorted(char_set)))

In [15]:
def create_batch(data, n=128):
    #Create a batch of n samples, each row in X representing SEQ_LEN letters from a song
    #with each row in y representing the one-hot encoding of the next letter (or the STOP character "S")
    #p_start determines the probability of starting at the beginning of the song vice a random point
    X = np.zeros((n, SEQ_LEN, K))
    y = np.zeros((n, K))
    
    for i in range(n):
        #random.choice(N) would make sequences ending in "E" SEQ_LEN times as likely
        #I still wanted them to be more common than uniform probability; here they are about 6x as likely
        song_idx = np.random.choice(N - int(SEQ_LEN * .9))
        song_len = len(data[song_idx])
        
        #We don't want to run out of song!  Clip the random choice to be within valid range
        start_idx = min(np.random.choice(song_len), song_len - SEQ_LEN - 1)
        
        #Iterate over letters in the song and one-hot encode them into the array
        for j, letter in enumerate(data[song_idx][start_idx:start_idx + SEQ_LEN]):
            letter_idx = letter2idx[letter]
            X[i, j, letter_idx] = 1
        
        #One-hot encode the next letter
        next_letter_idx = letter2idx[data[song_idx][start_idx + SEQ_LEN]]
        y[i, next_letter_idx] = 1
    
    return X, y

In [16]:
X, y = create_batch(data)

In [17]:
index_iter = iter(range(len(X)))

In [18]:
#Test to see if create_batch worked properly
i = next(index_iter)
"".join([idx2letter[idx] for idx in X[i].argmax(axis = 1)]), idx2letter[y[i].argmax()]

(" but you just wouldn't listen. now pay me no mind. so i'm movin'", ' ')

In [19]:
#Check what proportion of the next letters are the end of the song
np.mean(np.array([idx2letter[idx] for idx in y.argmax(axis = 1)]) == "E")

0.125

In [20]:
X.shape, y.shape

((128, 64, 46), (128, 46))

In [21]:
#Loading in the model I previously trained
model = load_model("models/songlyrics_gru_v1.hdf5")

In [22]:
"""
model = Sequential()
#return_sequences = True is required if plugging into another recurrent layer
model.add(GRU(128, dropout = .1, recurrent_dropout = .1, input_shape = (SEQ_LEN, K), return_sequences = True))
model.add(BatchNormalization())
model.add(GRU(128, dropout = .2, recurrent_dropout = .2))
model.add(BatchNormalization())
model.add(Dense(512, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(.3))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(.5))
model.add(Dense(K, activation = "softmax"))
""";

In [23]:
#Save model weights at the end of each epoch
#chk_callback = ModelCheckpoint("models/songlyrics_gru_v1.hdf5", save_best_only = True)
#Save logs to check out TensorBoard
#tb_callback = TensorBoard()

In [24]:
#model.compile("adam", "categorical_crossentropy", ["accuracy"])

In [25]:
#model.train_on_batch(X, y)

In [26]:
#X, y = create_batch(data, n = 100000)

In [27]:
#model.fit(X, y, batch_size = 128, epochs = 20, callbacks = [chk_callback, tb_callback])

In [28]:
def make_song(model, start = " " * SEQ_LEN):
    #model (fitted Keras model) 
    #start (str) : 
    #  Beginning of the song to continue filling in with predictions
    
    #Pad with whitespace regardless of what comes in, send to lower case
    #make_song will break if user inputs something not in the alphabet
    start = list((" " * SEQ_LEN + start).lower())
    
    #Get the index number for the final SEQ_LEN letters
    X_digits = [letter2idx[letter] for letter in start[-SEQ_LEN:]]
    X = []
    
    for digit in X_digits:
        #Create a one-hot encoding for each letter
        row = np.zeros(K)
        row[digit] = 1
        X.append(row)

    #While we haven't predicted the end character
    while start[-1] != "E":
        #predict the next character, grabbing the last SEQ_LEN rows from X
        #prediction returns an array of just one element, so we index to retrieve it
        pred = model.predict(np.array(X[-SEQ_LEN:]).reshape((1, SEQ_LEN, K)))[0]
        
        #Force it to make songs of a decent length by setting P("E") = 0 and normalizing
        if len(start) < 300:
            pred[letter2idx["E"]] = 0
            pred = pred / pred.sum()
            
        #Also returns an array
        prediction = np.random.choice(K, 1, p = pred)[0]
        row = np.zeros(K)
        row[prediction] = 1
        X.append(row)
        start.append(idx2letter[prediction])
    
    #Return the letters, strip out the whitespace padding and drop the "E" character
    #Possible TODO: Enforce proper capitalization on the resulting song
    return "".join(start).strip()[:-1]

In [29]:
#Left this running for a while and periodically checked the songs it was making
#I believe there's lots of room left for training, but it's so slow and I'm moving on!
for i in range(1):
    stats = model.train_on_batch(*create_batch(data))
    if i % 100 == 0:
        print("Iteration {}, {}".format(i, stats))
        print(make_song(model))
        model.save("models/songlyrics_gru_v2.hdf5")

Iteration 0, [1.2992927, 0.625]
every mother was a sweet it be bromone jost squin us. i've watch to have joze at the shoves. and i cause the damn, to take out. just love me when you really blied choose on me out for the keymorig highy and sun.


In [32]:
#Some starting lyrics:
song_index = np.random.choice(len(song_df))
print(song_df.text[song_index][:SEQ_LEN*3])
print()

#Grab up to SEQ_LEN*2 because the first SEQ_LEN chars are whitespace
print(make_song(model, start = song_df.clean[song_index][:SEQ_LEN*2]))

Last night my kisses were banked in black hair  
And in my bed, my lover, her hair was midnight black  
And all her mystery dwelled within her black hair  
And her black hair framed a happy he

last night my kisses were banked in black hair. and in my bed, my houndy famich? they make you, there this pappecky, me. who's not over too and wraved. clefived unything a smuch to gore. find me.


In [33]:
song_index = np.random.choice(len(song_df))
print(song_df.text[song_index][:SEQ_LEN*3])
print()

print(make_song(model, start = song_df.clean[song_index][:SEQ_LEN*2]))

Now if you wanna hear some boogie like I'm gonna play  
It's just an old piano and a knockout bass  
The drummer man's a cat they call Kickin' McCoy  
You know, remember that rubber-legged boy

now if you wanna hear some boogie like i'm gonna play. it's just die. your eyes. close they town to go to say. oh oh oh the lag his thing it's easy us, old by, kill to look away.


In [ ]:
#As we can see, it really comes up with some random stuff sometimes
#The model could use more work (additional training, more expressive),
#but I'd be curious to see how it'd perform given just one genre of music